# LlamaIndexでAgent

## 目次
- [概要](#概要)
- [参考](#参考)
- [チェック](#チェック)
- [準備](#準備)
- [最も簡単なエージェント](#最も簡単なエージェント)
- [エージェントにRAGを追加する](#エージェントにRAGを追加する)

## 概要
- LlamaIndex（公式）をトレースして基本的な利用方法を確認する。
- 破壊的に変更が発生するまで使えるでしょう。
- 破壊的に変更が発生後は、公式サイトの当該バージョンの情報（≒一次情報）をあたって。

## 参考

LlamaIndex - .NET 開発基盤部会 Wiki  
https://dotnetdevelopmentinfrastructure.osscons.jp/index.php?LlamaIndex

## チェック

In [1]:
#!pip list

In [2]:
#%env

## 準備

### ライブラリ読み込み

In [3]:
from llama_index.llms.openai import OpenAI
# from llama_index.llms.ollama import Ollama
from llama_index.core.agent import ReActAgent

### LLMの設定
エージェントでは揺らぐと困るので、temperature=0 に設定している。

#### OpenAI
OpenAIでないと動かない場合がありそう。

In [4]:
llm = OpenAI(model="gpt-3.5-turbo", temperature=0)

#### Ollama
- 「mixtral:8x7b」など大規模なモデルが必要らしい。
- Llama3の1Bなど、SLMでは動作しなかった。

In [5]:
#llm = Ollama(model="Llama3", temperature=0, request_timeout=360.0)

## 最も簡単なエージェント

### ライブラリ読み込み

In [6]:
from llama_index.core.tools import FunctionTool

### ファンクションツールのエージェント

#### ファンクションツールの定義

In [7]:
def multiply(a: float, b: float) -> float:
    """Multiply two numbers and returns the product"""
    return a * b

multiply_tool = FunctionTool.from_defaults(fn=multiply)

def add(a: float, b: float) -> float:
    """Add two numbers and returns the sum"""
    return a + b

add_tool = FunctionTool.from_defaults(fn=add)

#### エージェントの構成

In [8]:
agent = ReActAgent.from_tools([multiply_tool, add_tool], llm=llm, verbose=True)

#### エージェントの実行

In [9]:
response = agent.chat("What is 20 + (2 * 4) ? Use a tool to calculate every step.")
print(response)

> Running step 13c7dcfa-ea2e-4fbe-b4bc-00bf54511069. Step input: What is 20 + (2 * 4) ? Use a tool to calculate every step.
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: multiply
Action Input: {'a': 2, 'b': 4}
Observation: 8
> Running step 42d28cd9-aedf-4656-bd4a-fbf7b90693dc. Step input: None
Thought: I need to add 20 to the result of the multiplication.
Action: add
Action Input: {'a': 20, 'b': 8}
Observation: 28
> Running step ed30071d-7c7d-4a80-b978-43cda3a5eef2. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: The result of 20 + (2 * 4) is 28.
The result of 20 + (2 * 4) is 28.


## エージェントにRAGを追加する

### 準備

#### 使用する変数

In [10]:
DATA_DIR = "./llamaindex/data/2023_canadian_budget"
PERSIST_DIR = "./llamaindex/storage/2023_canadian_budget"
CHROMA_DIR = "./llamaindex/chroma_db/2023_canadian_budget"

#### ライブラリ読み込み

In [11]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
#from llama_index.embeddings.huggingface import HuggingFaceEmbedding

#### 簡単なテスト（永続化付き）

##### Settings

In [12]:
# bge-base embedding model
#Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")
Settings.llm = llm # 複数箇所で使うのでグローバルに設定

##### Indexing & Storing

In [13]:
import os.path
from llama_index.core import (
    StorageContext,
    load_index_from_storage,
)

# check if storage already exists
if not os.path.exists(PERSIST_DIR):
    # load the documents and create the index
    documents = SimpleDirectoryReader(DATA_DIR).load_data()
    index = VectorStoreIndex.from_documents(documents)
    # store it for later
    index.storage_context.persist(persist_dir=PERSIST_DIR)
else:
    # load the existing index
    storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR)
    index = load_index_from_storage(storage_context)

##### Querying

In [14]:
# Either way we can now query the index
query_engine = index.as_query_engine()
response = query_engine.query(
    "What was the total amount of the 2023 Canadian federal budget?"
    "\n Please answer with one number."
)
print(response)

$496.9 billion


#### ツール作成

##### ファンクション・ツール

In [15]:
from llama_index.core.tools import FunctionTool

In [16]:
# function tools
def multiply(a: float, b: float) -> float:
    """Multiply two numbers and returns the product"""
    return a * b

multiply_tool = FunctionTool.from_defaults(fn=multiply)

def add(a: float, b: float) -> float:
    """Add two numbers and returns the sum"""
    return a + b

add_tool = FunctionTool.from_defaults(fn=add)

##### クエリエンジンツール

In [17]:
from llama_index.core.tools import QueryEngineTool

In [18]:
budget_tool = QueryEngineTool.from_defaults(
    query_engine,
    name="canadian_budget_2023",
    description="A RAG engine with some basic facts about the 2023 Canadian federal budget.")

#### エージェントにツールを実行させる

##### ライブラリ読み込み

In [19]:
from llama_index.core.agent import ReActAgent

##### RAG込エージェント定義

In [20]:
agent = ReActAgent.from_tools([multiply_tool, add_tool, budget_tool], verbose=True)

##### RAG込エージェント実行
- プロンプトの指示の通り、budget_toolのRAGから予算を読み込んで3倍している。
- multiply_tool, add_toolは渡してあるが、使用しないで回答できている。
- （チュートリアルでは、multiply_toolを使う予定ではあったものの）
- SLMだと上手く動作しない問題。そしてデバッグと対策も不明（笑）

In [21]:
response = agent.chat("What is the total amount of the 2023 Canadian federal budget multiplied by 3?")
print(response)

> Running step f9980254-0d8e-4cee-a89d-7f649913c042. Step input: What is the total amount of the 2023 Canadian federal budget multiplied by 3?
Thought: The current language of the user is English. I need to use a tool to help me answer the question.
Action: canadian_budget_2023
Action Input: {'input': 'total'}
Observation: $496.9 billion
> Running step 6ee94b63-8908-42e1-a6a0-f685b930a866. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: The total amount of the 2023 Canadian federal budget multiplied by 3 is $1.49 trillion.
The total amount of the 2023 Canadian federal budget multiplied by 3 is $1.49 trillion.
